# Predicción de Stocks con LSTM Bayesiano (con PyTorch)

### Librerías

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import plotly.express as px

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

### Llamado de datos

In [2]:
tabla='2828HK'
conn = mysql.connector.connect(user='root', password='', host='localhost', database='stock_exchange')
stock = pd.DataFrame(pd.read_sql("SELECT * FROM "+tabla, conn))
conn.close()
stock.head(5)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,index,Date,Open,High,Low,Close,Volume
0,0,2019-01-02,102.800003,102.800003,99.250000,99.500000,12118897
1,1,2019-01-03,99.949997,100.400002,98.849998,99.599998,3562140
2,2,2019-01-04,99.000000,101.699997,99.000000,101.300003,2854212
3,3,2019-01-07,102.400002,103.300003,102.000000,102.300003,3527707
4,4,2019-01-08,102.699997,103.199997,102.000000,102.400002,4159683


### Selección y Preparación de Datos

In [3]:
Selecccionadas=['Date', 'Open']
stock_2=stock[Selecccionadas]
stock_2=stock_2.loc[(stock_2["Date"] >= '2022-06-01')]
stock_2.head(5)

,Date,Open
840,2022-06-01,75.300003
841,2022-06-02,73.800003
842,2022-06-06,74.139999
843,2022-06-07,76.139999
844,2022-06-08,76.400002


In [4]:
stock_2['Open']=np.log(stock_2['Open'])
stock_2.head(5)

,Date,Open
840,2022-06-01,4.321480
841,2022-06-02,4.301359
842,2022-06-06,4.305955
843,2022-06-07,4.332574
844,2022-06-08,4.335983


In [5]:
plot_length = 150
plot_df = stock_2.copy(deep=True).iloc[:plot_length]
plot_df['Date'] = plot_df['Date']

fig = px.line(plot_df,
              x="Date",
              y="Open",
              title="Log de Open Stock vs Time")
fig.update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)',
    'yaxis.color' : 'white',
    'xaxis.color' : 'white',
    'title_font_color' : 'white'
})
fig.update_xaxes(showgrid=False, zeroline=True)
fig.update_yaxes(showgrid=False, zeroline=True)
fig.show()

### División en Entrenamiento y Prueba

In [6]:
from sklearn.preprocessing import MinMaxScaler

def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)

train_split = 0.7
n_train = int(train_split * len(stock_2))
n_test = len(stock_2) - n_train

features = ['Open']
feature_array = stock_2[features].values

# Fit Scaler only on Training features
feature_scaler = MinMaxScaler()
feature_scaler.fit(feature_array[:n_train])
# Fit Scaler only on Training target values
target_scaler = MinMaxScaler()
target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

# Transfom on both Training and Test data
scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                            columns=features)

sequence_length = 10
X, y = create_sliding_window(scaled_array, 
                             sequence_length)

X_train = X[:n_train]
y_train = y[:n_train]

X_test = X[n_train:]
y_test = y[n_train:]

### Arquitectura del Modelo Bayesiano LSTM

In [7]:
class BayesianLSTM(nn.Module):
    
    def __init__(self, n_features, output_length, batch_size):

        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size # user-defined

        self.hidden_size_1 = 128 # number of encoder cells (from paper)
        self.hidden_size_2 = 32 # number of decoder cells (from paper)
        self.stacked_layers = 2 # number of (stacked) LSTM layers for each stage
        self.dropout_probability = 0.5 # (Girar) arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features, 
                             self.hidden_size_1, 
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        
        self.fc = nn.Linear(self.hidden_size_2, output_length) #red densa
        self.loss_fn = nn.MSELoss()
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :] # take the last decoder cell's outputs
        y_pred = self.fc(output)
        #y_predic(gaus)=.....
        return y_pred # si se asigna una distribución gaussina
        
    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state
    
    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state
    
    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

### Entrenamiento del Modelo

In [8]:
n_features = scaled_array.shape[-1]
sequence_length = 10
output_length = 1

batch_size = 128
n_epochs = 150
learning_rate = 0.01

bayesian_lstm = BayesianLSTM(n_features=n_features,
                             output_length=output_length,
                             batch_size = batch_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)

In [9]:
bayesian_lstm.train()

for e in range(1, n_epochs+1):
    for b in range(0, len(X_train), batch_size):
        features = X_train[b:b+batch_size,:,:]
        target = y_train[b:b+batch_size]    

        X_batch = torch.tensor(features,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)

        output = bayesian_lstm(X_batch)
        loss = criterion(output.view(-1), y_batch)  

        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 

    if e % 10 == 0:
      print('epoch', e, 'loss: ', loss.item())

epoch 10 loss:  0.0064329179003834724
epoch 20 loss:  0.00986534170806408
epoch 30 loss:  0.010260174050927162
epoch 40 loss:  0.013815293088555336
epoch 50 loss:  0.005792480893433094
epoch 60 loss:  0.013271518982946873
epoch 70 loss:  0.01093522273004055
epoch 80 loss:  0.012505828402936459
epoch 90 loss:  0.011590724810957909
epoch 100 loss:  0.013019182719290257
epoch 110 loss:  0.008352152071893215
epoch 120 loss:  0.009628759697079659
epoch 130 loss:  0.009063641540706158
epoch 140 loss:  0.07308775931596756
epoch 150 loss:  0.012484550476074219


### Evaluación del Rendimiento del Modelo

In [10]:
offset = sequence_length

def inverse_transform(y):
  return target_scaler.inverse_transform(y.reshape(-1, 1))

training_df = pd.DataFrame()
training_df['Date'] = stock_2['Date'].iloc[offset:n_train + offset:1] 
training_predictions = bayesian_lstm.predict(X_train)
training_df['Open'] = inverse_transform(training_predictions)
training_df['source'] = 'Training Prediction'

training_truth_df = pd.DataFrame()
training_truth_df['Date'] = training_df['Date']
training_truth_df['Open'] = stock_2['Open'].iloc[offset:n_train + offset:1] 
training_truth_df['source'] = 'True Values'

testing_df = pd.DataFrame()
testing_df['Date'] = stock_2['Date'].iloc[n_train + offset::1] 
testing_predictions = bayesian_lstm.predict(X_test)
testing_df['Open'] = inverse_transform(testing_predictions)
testing_df['source'] = 'Test Prediction'

testing_truth_df = pd.DataFrame()
testing_truth_df['Date'] = testing_df['Date']
testing_truth_df['Open'] = stock_2['Open'].iloc[n_train + offset::1] 
testing_truth_df['source'] = 'True Values'

evaluation = pd.concat([training_df, 
                        testing_df,
                        training_truth_df,
                        testing_truth_df
                        ], axis=0)

In [11]:
fig = px.line(evaluation.loc[evaluation['Date'].between('2019-01-02', '2023-03-24')],
                 x="Date",
                 y="Open",
                 color="source",
                 title="Log of Open vs Time")
fig.show()

### Cuantificación de la Incertidumbre

In [12]:
n_experiments = 100

test_uncertainty_df = pd.DataFrame()
test_uncertainty_df['Date'] = testing_df['Date']

for i in range(n_experiments):
  experiment_predictions = bayesian_lstm.predict(X_test)
  test_uncertainty_df['log_Open_{}'.format(i)] = inverse_transform(experiment_predictions)

log_energy_consumption_df = test_uncertainty_df.filter(like='log_Open', axis=1)
test_uncertainty_df['log_Open_mean'] = log_energy_consumption_df.mean(axis=1)
test_uncertainty_df['log_Open_std'] = log_energy_consumption_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['Date', 'log_Open_mean', 'log_Open_std']]

C:\Users\rodrigo.gomez\AppData\Local\Temp\ipykernel_5212\2106607846.py:8: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [13]:
test_uncertainty_df['lower_bound'] = test_uncertainty_df['log_Open_mean'] - 3*test_uncertainty_df['log_Open_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['log_Open_mean'] + 3*test_uncertainty_df['log_Open_std']

In [14]:
import plotly.graph_objects as go

test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['Date'].between('2019-01-02', '2023-03-24')]
truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['Date'].between('2019-01-02', '2023-03-24')]

upper_trace = go.Scatter(
    x=test_uncertainty_plot_df['Date'],
    y=test_uncertainty_plot_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )
lower_trace = go.Scatter(
    x=test_uncertainty_plot_df['Date'],
    y=test_uncertainty_plot_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.1)',
    name='99% Lower Confidence Bound'
    )
real_trace = go.Scatter(
    x=truth_uncertainty_plot_df['Date'],
    y=truth_uncertainty_plot_df['Open'],
    mode='lines',
    fill=None,
    name='Real Values'
    )

data = [upper_trace, lower_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Cuantificación de incertidumbre para datos de prueba Para el valor Open',
                   xaxis_title='Tiempo',
                   yaxis_title='log_Open (USD)')

fig.show()

#### Evaluación de la Incertidumbre

In [15]:
bounds_df = pd.DataFrame()

# Using 99% confidence bounds
bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
bounds_df['prediction'] = test_uncertainty_plot_df['log_Open_mean']
bounds_df['real_value'] = truth_uncertainty_plot_df['Open']
bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                          (bounds_df['real_value'] <= bounds_df['upper_bound']))

print("Proportion of points contained within 99% confidence interval:", 
      bounds_df['contained'].mean())

Proportion of points contained within 99% confidence interval: 0.6274509803921569


## Candlestick

In [16]:
stock_3=stock.loc[(stock["Date"] >= '2023-01-10')]
fig2 = go.Figure(data=[go.Candlestick(x=stock_3['Date'],
                       open=stock_3['Open'], high=stock_3['High'],
                       low=stock_3['Low'], close=stock_3['Close'])])

fig2.show()